<a href="https://colab.research.google.com/github/edoardochiarotti/class_datascience/blob/main/2023/05_Mean-Model/Resources/05_Mean_model.ipynb" target="_blank" rel="noopener"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Gentle Introduction to the Linear Regression Model: The Mean Model

<img src="https://i.imgflip.com/82uqjd.jpg" width="500">

In [ ]:
# PACKAGES
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statistics as st
import statsmodels.api as sm

# FUNCTIONS FROM PACKAGES
from numpy.linalg import inv

# SEABORN THEME
scale = 0.4
W = 16*scale
H = 9*scale
sns.set(rc = {'figure.figsize':(W,H)})
sns.set_style("white")

Main References and notation:
- For more theory in statistics and econometrics, we relied on various sources on the Web and the following textbooks: 
    - J. Wooldridge, Econometric Analysis of Cross Section and Panel Data, MIT Press, 2002
    - William H. Greene, Econometric Analysis, sixth edition, Pearson.
- We will denote N-dimensional column vectors (of random variables) with lower bold letters, i.e. $\boldsymbol{y} = (y_1,...,y_N)'$, and matrixes (of random variables) with capital bold letters, i.e. $\bf{Y}$.

## Content
- [The Multivariate Normal Distribution](#The-Multivariate-Normal-Distribution)
- [Marginal and Conditional Probability Distributions](#Marginal-and-Conditional-Probability-Distributions)
- [The Mean Model](#The-Mean-Model)
- [The Sample Mean and Variance](#The-Sample-Mean-and-Variance)
- [The Mean Model in Matrix Notation](#The-Mean-Model-in-Matrix-Notation)
- [The Sample Mean and Variance in Matrix Notation](#The-Sample-Mean-and-Variance-in-Matrix-Notation)

- Before checking the mean model (and so regression), let's re-fresh our knowledge of multivariate density functions.

## The Multivariate Normal Distribution <a name="The-Multivariate-Normal-Distribution"></a>
- The **multivariate normal distribution**, multivariate Gaussian distribution, or joint normal distribution is a generalization of the one-dimensional (univariate) normal distribution to higher dimensions ([Wikipedia, Multivariate Normal Distribution](https://en.wikipedia.org/wiki/Multivariate_normal_distribution)).
- The multivariate normal distribution is often used to describe, at least approximately, any set of (possibly) correlated real-valued random variables each of which clusters around a mean value.
- Let's take a 2-dimensional random vector $\boldsymbol{y}=(y_1, y_2)'$. Remember, $y_1$ is a random variable and $y_2$ is a random variable.
- Let's write down the **probability density function** for the joint normal distribution:
<br><br>
$$
f(y_1,y_2)=\frac{1}{2\pi\sigma_1 \sigma_2 \sqrt{1-\rho^2}}\exp\left(-\frac{1}{2(1-\rho^2)}\left[\left(\frac{y_1-\beta_1}{\sigma_1}\right)^2 -2\rho\ \left(\frac{y_1-\beta_1}{\sigma_1}\right) \left(\frac{y_2-\beta_2}{\sigma_2}\right) + \left(\frac{y_2-\beta_2}{\sigma_2}\right)^2 \right]\right)
$$
<br>
- We can write the same in matrix form:
<br><br>
$$
f(\boldsymbol{y})=\frac{1}{\sqrt{(2\pi)^2 |\boldsymbol{\Sigma}|}}\exp\left(-\frac{1}{2}(\boldsymbol{y}-\boldsymbol{\beta})'\boldsymbol{\Sigma}^{-1}(\boldsymbol{y}-\boldsymbol{\beta})\right)
$$
<br>
- Stop for a moment. Write down these equations with pen and paper, and below each vector / matrix, write down its dimensions. For example, if a matrix has 2 columns and 2 rows, write $2 \times 2$ below the matrix. As you do that, check that all dimensions allow matrix multiplication (i.e. matrixes are conformable). 
- The **moments** of the distribution are the following (I omit the formulas with the integrals here):
<br><br>
$$
\mathrm{E}(\boldsymbol{y})=
\begin{bmatrix}
\mathrm{E}(y_1) \\
\mathrm{E}(y_2)
\end{bmatrix} =
\begin{bmatrix}
\beta_1 \\
\beta_2
\end{bmatrix}=\boldsymbol{\beta}
$$
<br>
$$
\mathrm{V}(\boldsymbol{y})=E[(\boldsymbol{y}-\boldsymbol{\beta})(\boldsymbol{y}-\boldsymbol{\beta})']= 
\begin{bmatrix}
\sigma_1^2 & \rho \\
\rho & \sigma_2^2 
\end{bmatrix} = 
\boldsymbol{\Sigma}
$$
<br>
- We can express all of this in the usual coincise notation:
<br>
$$
\boldsymbol{y} \sim \mathcal{N}(\boldsymbol{\beta},\,\boldsymbol{\Sigma})
$$
<br>

<img src="https://i.imgflip.com/82y3gm.jpg" width="500">

## Marginal and Conditional Probability Distributions <a name="Marginal-and-Conditional-Probability-Distributions"></a>
- It is important to distinguish between the joint probability distribution of $y_1$ and $y_2$ and the probability distribution of each variable individually. The individual probability distribution of a random variable is referred to as its **marginal probability distribution**. In general, the marginal probability distribution of $y_1$ can be determined from the joint probability distribution of $y_1$ and $y_2$. Let's write it down, and let's express its distributions and moments already in the coincise notation:
<br><br>
$$
f(y_1) = \int f(\boldsymbol{y})dy_2 = \frac{1}{\sigma_1\sqrt{2\pi}}\exp\left(-\frac{1}{2}\left(\frac{y_1-\beta_1}{\sigma_1}\right)^{2}\right)
$$
<br>
$$
y_1 \sim \mathcal{N}(\beta_1,\,\sigma_1^2)
$$
<br>
- Another imporatnt concept is the **conditional probability distribution**. Given two jointly distributed random variables $y_1$ and $y_2$, the conditional probability distribution of $y_1$ given $y_2$ is the probability distribution of $y_1$ when $y_2$ is known to be a particular value ([Wikipedia, Conditional Probability Distribution](https://en.wikipedia.org/wiki/Conditional_probability_distribution)). The conditional distribution contrasts with the marginal distribution of a random variable, which is its distribution without reference to the value of the other variable.
- Let's refer to $y_1$ conditional on $y_2$ as $y_1|y_2$. $y_1|y_2$ is a random variable, which has it own density function, i.e. the conditional density function, and its moments, which we can express with the usual coincise notation:
<br><br>
$$
f(y_1|y_2)=\frac{f(\boldsymbol{y})}{f(y_2)}=\frac{\frac{1}{\sqrt{(2\pi)^2 |\Sigma|}}\exp\left(-\frac{1}{2}(\boldsymbol{y}-\boldsymbol{\beta})'\boldsymbol{\Sigma}^{-1}(\boldsymbol{y}-\boldsymbol{\beta})\right)
}{\frac{1}{\sigma_2\sqrt{2\pi}}\exp\left(-\frac{1}{2}\left(\frac{y_2-\beta_2}{\sigma_2}\right)^{2}\right)}
$$
<br>
$$
y_1|y_2 \sim \mathcal{N}\left(\beta_1 + \frac{\sigma_1}{\sigma_2}\rho(y_2-\beta_2), \, (1-\rho)^2\sigma_1^2\right)
$$
<br>

- So far we have considered a random vector of 2 random variables. So what we have seen is more precisely the bivariate normal distribution and its marginals and conditionals. All these formulas can be **generalized**. 
- For example, when we have a N-dimensional random vector $\boldsymbol{y}=(y_1, y_2, ..., y_N)'$ that is normally distributed with mean $\boldsymbol{\beta}$ and variance-covariance matrix $\boldsymbol{\Sigma}$, we say that it follows a multivariate normal distribution $\boldsymbol{y} \sim \mathcal{N}(\boldsymbol{\beta},\,\boldsymbol{\Sigma})$. Note that the matrix notation we used it's already generalized, so (almost) all of the above holds for N-dimensional random vectors.
- Also note that conditional distributions can be generalized for random vectors and matrixes (rather than random variables). For example we can have a N-dimensional random vector $\boldsymbol{\epsilon}$ and a $N \times M$ matrix $\boldsymbol{y}$, and we can express the random vector $\boldsymbol{\epsilon}$ conditional on the random matrix $\boldsymbol{y}$ as $\boldsymbol{\epsilon}|\boldsymbol{y}$.
- For more notation on the Multivariate Normal Distribution, you can check [Wikipedia - Multivariate Normal Distribution](https://en.wikipedia.org/wiki/Multivariate_normal_distribution). For a reference in Python, you can check how it's done with `Numpy` [here](https://numpy.org/doc/stable/reference/random/generated/numpy.random.multivariate_normal.html). The functions are similar to the univariate case.

<img src="https://i.imgflip.com/82ybm1.jpg" width="500">

## The Mean Model <a name="The-Mean-Model"></a>
- Let's now re-consider the concepts of population model, random sample, and model of the Data Generating Process (DGP).
- The **population model** assumes that the population of the variable of interest (e.g., CO2 emissions per capita) can be modeled with a probability distribution, which in our case is normal with mean $\beta$ and variance $\sigma^2$:
<br><br>
$$y\sim N(\beta,\sigma^2)$$
<br>
- As we can't observe the population, we need need to work with a sample. We have said that, before drawing the actual sample, we can refer to our hypothetical sample as the **random sample**. The random sample is a set of random variables, each of which represents the hypothetical value that the respective observation of the sample can take up: 
<br><br>
$$\{y_1,...,y_N\}$$
<br>
- The **model of the DGP** assumes that each observation of the random sample, $y_i$, is generated by an underlying process, i.e. the DGP, described by the distribution of the population, which is a normal. In addition, it assumes that these distributions are independent from each other. Overall, we can say that in the model $y_i$ are identical and independently distributed (i.i.d.):
<br><br>
$$y_i \sim \mathcal{N}(\beta,\,\sigma^2), \, \,  \text{for }i=1,...,N$$
<br>
- The model of the DGP can be re-written in linear-regression form (exact same thing). When we write it in linear-regression form (again, exact same thing, just different notation), we refer to it as the **mean model**:
<br><br>
$$y_i = \beta + \epsilon_i \qquad \text{with } \epsilon_i \sim \mathcal{N}(0,\,\sigma^2), \, \, \text{for } i=1,...,N$$
<br>
- We can say that we have split $y_i$ in its fixed mean component, $\beta$, and its random component, $\epsilon_i$. But the model is exactly the same, nothing has changed. It's just another way of expressing it.

## The Sample Mean and Variance <a name="The-Sample-Mean-and-Variance"></a>
- As we have seen in the previous class, the **sample mean** is a good estimator for $\beta$. As we are just re-writing the model of the DGP in regression form, everything that we have seen applied here (just there will be some new terminology to learn, stemming from the fact that now we are using the regression form). So let's re-write the formulas of the sample-mean estimator in compact form:
<br><br>
$$
\hat{\beta}_{SM}=\frac{1}{N}\sum_{i=1}^{N}y_i
$$
<br>
$$
\hat{\beta}_{SM} \sim \mathcal{N}(\beta, \, \frac{\sigma^2}{N})
$$
<br>
- Here's the new terminology. Let's now define the predicted (or fitted) values and the residuals of our model as follows:
<br><br>
$$
\hat{y}_i = \hat{\beta}_{SM} \qquad \text{for } i=1,...,N
$$
<br>
$$
\hat{\epsilon}_i = y_i - \hat{y}_i = y_i - \hat{\beta}_{SM} \qquad \text{for } i=1,...,N
$$
<br>
- Great. Also from the previous class we know that the (population) **sample variance** is a good estimator for $\sigma^2$, which we can then use to estimate the standard error of the mean:
<br><br>
$$
\hat{\sigma}_{SV}^2=\frac{1}{N-1}\sum_{i=1}^{N}(y_i-\hat{\beta}_{SM})^2
$$
<br>
$$
\frac{N-1}{\sigma^2}\hat{\sigma}_{SV}^2 \sim \mathcal{\chi}^2_{N-1}
$$
<br>

## The Mean Model in Matrix Notation <a name="The-Mean-Model-in-Matrix-Notation"></a>
- OK so we have the mean model and we have our estimators to estimate the population's parameters of interest. So far we have expressed the mean model in terms of our random variables $y_i$, which form our random sample. The random sample can actually be expressed as an N-dimensional vector of random variables $\boldsymbol{y}=(y_1,y_2,...,y_N)'$. So instead of writing the model of the DGP as we did above, using "for $i=1,...,N$" to indicate that that holds for each future observation of the random sample, we can express the model of the DGP using matrix notation. Let's define a N-dimensional vector of 1s $\boldsymbol{x}=(1,...,1)'$ and write our model of the DGP with **matrix notation**:
<br><br>
$$
\boldsymbol{y} \sim \mathcal{N}(\boldsymbol{x}\beta, \, \sigma^2 \boldsymbol{I}_N)
$$
<br>
- Think for a second about what this is and means. We said that each random variable $y_i$ of the random sample follows a distribution, which is the one of the population (they are identically distributed). So when you take all the random variables $y_i$ together in a random sample, i.e. $(y_1,...,y_N)$ it's like having N distributions that are all the same, with same mean $\beta$ and variance $\sigma^2$. Yeah, indeed. So you have $N$ $betas$ and $N$ $sigmas$. 
- The mean model, i.e. the model of the DGP in regression form, can therefore be expressed as follows (always with $\boldsymbol{x}=(1,,...,1)'$):
<br><br>
$$
\boldsymbol{y} = \boldsymbol{x}\beta + \boldsymbol{\epsilon} \qquad \text{with } \boldsymbol{\epsilon} \sim \mathcal{N}(\boldsymbol{0}_N,\,\sigma^2\boldsymbol{I}_N)
$$
<br>
- The model we just wrote says that the random vector $\boldsymbol{y}$ follows a N-variate normal distribution, with **mean vector and variance-covariance matrix of the model** as follows:
<br><br>
$$
\mathrm{E}(\boldsymbol{y})= \mathrm{E}(\boldsymbol{x}\beta+\boldsymbol{\epsilon}) = \mathrm{E}(\boldsymbol{x}\beta) = \boldsymbol{x}\beta =
\begin{bmatrix}
1 \\
\vdots \\
1
\end{bmatrix}\beta=
\begin{bmatrix}
\beta \\
\vdots \\
\beta
\end{bmatrix}
$$
<br>
$$
\mathrm{V}(\boldsymbol{y}) = \mathrm{V}(\boldsymbol{x}\beta+\boldsymbol{\epsilon}) = \mathrm{V}(\boldsymbol{\epsilon}) = \sigma^2\boldsymbol{I}_N = \sigma^2
\begin{bmatrix}
1 & 0 & \dots & 0 \\
0 & 1 & \dots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \dots & 1 \\
\end{bmatrix}=
\begin{bmatrix}
\sigma^2 & 0 & \dots & 0 \\
0 & \sigma^2 & \dots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \dots & \sigma^2 \\
\end{bmatrix}
$$
<br>
- The first **element** $\beta$ in $\mathrm{E}(\boldsymbol{y})$ is the mean (first moment) of the distribution of $y_1$. Similarly, the first **diagonal element** $\sigma^2$ of $\mathrm{V}(\boldsymbol{y})$ is the variance (second moment) of the distribution of $y_1$. Again, you can think that this distribution is like the distribution of the population. Following a similar logic, the last $\beta$ in $\mathrm{E}(\boldsymbol{y})$ is the mean of the distribution of $y_N$, and the last diagonal element $\sigma^2$ of $\mathrm{V}(\boldsymbol{y})$ is the variance of the distribution of $y_N$. Again, you can think that this distribution is like the distribution of the population.
- So you have **N distributions** that are like the distribution of the population. Which is the same of saying that you have N random variables that follow the distribution of the population. Let's keep thinking about N distributions rather than 1 distribution, it's easier to then understand the size of our matrixes.
- So, indeed, what is the **size** of the mean and variance-covariance matrixes? As you can see, $\mathrm{E}(\boldsymbol{y})$ is $N\times1$ and $\mathrm{V}(\boldsymbol{y})$ is $N\times N$. You should write these matrixes down with pen and paper, and write their sizes under each matrix. Only through that ...

<img src="https://i.imgflip.com/82uw3k.jpg" width="500">

- Also note that the **off-diagonal elements** of $\mathrm{V}(\boldsymbol{y})$ are correlation coefficients, that express the correlation between two random variables $y_i$ and $y_j$, i.e. $\mathrm{Cov}(y_i,y_j)$ with $j\neq i$. As we are assuming that the variables are independent (aside from identically distributed), their correlation coefficients are all zero.

## The Sample Mean and Variance in Matrix Notation <a name="The-Sample-Mean-and-Variance-in-Matrix- Notation"></a>
- OK great. As we did for the mean model, we can also express our **estimators** for the mean and variance, and predicted values and residuals, in matrix form. Before checking what comes below, you should take a moment to think how we can do that. So, how can we express, say, the sample mean estimator $\frac{1}{N}\sum_{i=1}^{N}y_i$ in matrix form, knowing that we have defined the N-dimensional vector of 1s $\boldsymbol{x}=(1,...,1)'$? 
- Yeah we are gonna tell you. One way of doing it for sample mean, predicted values, residuals and variance is the following one:
<br><br>
$$
    \hat{\beta}_{SM}=\frac{1}{N}\sum_{i=1}^{N}y_i = (\boldsymbol{x}'\boldsymbol{x})^{-1}(\boldsymbol{x}'\boldsymbol{y})
$$
<br>
$$
\hat{\boldsymbol{y}}=\boldsymbol{x}\hat{\beta}_{SM}
$$
<br>
$$
\hat{\boldsymbol{\epsilon}}=\boldsymbol{y} - \hat{\boldsymbol{y}}=\boldsymbol{y} - \boldsymbol{x}\hat{\beta}_{SM}
$$
<br>
$$
\hat{\sigma}_{SV}^2=\frac{1}{N-1}\sum_{i=1}^{N}(y_i-\hat{\beta}_{SM})^2=\frac{(\boldsymbol{y}-\boldsymbol{x}\hat{\beta}_{SM})'(\boldsymbol{y}-\boldsymbol{x}\hat{\beta}_{SM})}{N-1}=\frac{\hat{\boldsymbol{\epsilon}}\,'\hat{\boldsymbol{\epsilon}}}{N-1}
$$
<br>
- You should take a moment to write down these equations and work out the results using your knowledge of matrix multiplication. For example, you should see why $(\boldsymbol{x}'\boldsymbol{x})=N$. As per usual, you should write the size of all vectors and matrixes below them. As you do that, you should notice that $\hat{\beta}_{SM}$ and $\hat{\sigma}_{SV}^2$ remain obviously one-dimensional random variables, which follow the usual univariate distributions (you can write down their distributions and moments with the usual coincise notation).
- OK so to **recap**, in the previous class we have introduced the concepts of population model, random sample, model of the DGP, and the sample-mean and sample-variance estimators. In this class, we have seen how we can express the model of the GDP in regression form - which we called mean model -, and how we can use matrixes to have a compact notation for the random sample and mean model. After expressing our model in regression form with matrix notation, we have also expressed the sample-mean and sample-variance estimators in matrix notation. We have done so because this is the most general notation you can have, and it will be super handy when we do the OLS estimator, with whatever number of regressors.